In [1]:
import pandas as pd
import numpy as np
from random import sample

In [2]:
populacao = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

## Amostragem aleatória simples

Criando amostras distintas para uma população fixa

In [7]:
amostra1 = sample(populacao, 4)
amostra2 = sample(populacao, 4)
amostra3 = sample(populacao, 4)

print("Amostra 1:", amostra1, "\nAmostra 2:", amostra2, "\nAmostra 3:", amostra3)

Amostra 1: [6, 4, 7, 2] 
Amostra 2: [8, 11, 7, 12] 
Amostra 3: [2, 9, 10, 7]


O mesmo pode ser executado usando a biblioteca Pandas

In [14]:
df = pd.DataFrame(populacao, columns = ['Valores'])

In [11]:
df.sample(n = 4)

,Valores
4,5
1,2
2,3
10,11


Ou pode-se usar o parâmetro *frac* para determinar a partir de uma fração do número total de elementos

In [13]:
df.sample(frac = 0.33)

,Valores
7,8
5,6
4,5
0,1


## Amostragem aleatória sistemática

Usando python "puro", ou sem biblioteca:

In [21]:
tamanho_populacao = len(populacao)

amostra_sistematica = []

for i in range(1, tamanho_populacao, 3):
  amostra_sistematica.append(populacao[i])

amostra_sistematica

[2, 5, 8, 11]

Utilizando o *numpy* para obter o mesmo resultado:

In [23]:
np.arange(1, 12, step = 3)

array([ 1,  4,  7, 10])

In [26]:
def amostragem_sistematica(df, inicio = 0, step = 3):
  indexes = np.arange(inicio, df.shape[0], step = step)
  amostra = df.iloc[indexes]
  return amostra

amostragem_sistematica(df)

,Valores
0,1
3,4
6,7
9,10


## Amostragem estratificada

In [27]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
b = ["A", "A", "A", "A", "B", "B", "C", "C", "C", "D", "D", "D"]

df = pd.DataFrame({"Grupo": b, "Valor": a})
df

,Grupo,Valor
0,A,1
1,A,2
2,A,3
3,A,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
9,D,10


In [34]:
def amostra_estratificada_1(df, n, estrato):
  amostra = df.groupby(estrato, group_keys = False).apply(lambda x: x.sample(min(len(x), n)))
  return amostra

Criando amostras estratificadas com tamanho 2 (sendo esse valor pelo fato do menor grupo dentro os dados ter o tamanho 2):

In [36]:
amostra_estratificada_1(df, 2, 'Grupo')

,Grupo,Valor
1,A,2
0,A,1
5,B,6
4,B,5
8,C,9
7,C,8
11,D,12
9,D,10


Caso seja solicitado um estrato de tamanho 3, não haverão erros nos grupos A, C e D, dado que todos tem tamanho maior ou igual a 3. Porém, o grupo B não poderá apresentar 3 itens dado que o mesmo possui somente 2. Nesse grupo seria esperado um erro de tamanho, mas temos na função um tratamento de erro para considerar o menor valor entre o tamanho do grupo e o número solicitado, de modo que grupos com menos elementos que o solicitado entregarão sempre todos os seus elementos, mesmo que com isso ele fique sub representado no resultado final

In [37]:
amostra_estratificada_1(df, 3, 'Grupo')

,Grupo,Valor
2,A,3
0,A,1
1,A,2
4,B,5
5,B,6
6,C,7
8,C,9
7,C,8
10,D,11
11,D,12


Existe aqui um problema: do modo feito acima, grupos maiores na população terão a mesma representatividade na amostra. Isso pode ser um problema em alguns casos, onde a proporção de elementos de cada grupo deve ser mantida dentro da amostra para que essa seja mais representativa da população. Nesse caso, é necessário que criemos uma nova função para considerar essa proporção na hora de selecionar os elementos.

In [40]:
def amostra_estratificada_2(df, n, estrato):
  tamanho_populacao = df.shape[0]
  amostra = df.groupby(estrato, group_keys = False)\
            .apply(lambda x: x.sample(int(n * len(x) / tamanho_populacao)))\
            .reset_index(drop = True)\
            .sort_values(by = estrato)

  return amostra

In [41]:
amostra_estratificada_2(df, 9, 'Grupo')

,Grupo,Valor
0,A,1
1,A,3
2,A,2
3,B,6
4,C,7
5,C,9
6,D,12
7,D,10


## Amostragem por conglomerado

In [44]:
def amostra_conglomerado(df, n_conglomerados, conglomerado):
  todos_conglomerados = list(df[conglomerado].unique())
  tamanho_conglomerados = len(todos_conglomerados)
  n = min(n_conglomerados, tamanho_conglomerados)
  conglomerados_sorteados = sample(todos_conglomerados, n)

  amostra = df[df[conglomerado].isin(conglomerados_sorteados)]
  return amostra

In [45]:
amostra_conglomerado(df, 2, 'Grupo')

,Grupo,Valor
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [46]:
amostra_conglomerado(df, 2, 'Grupo')

,Grupo,Valor
4,B,5
5,B,6
9,D,10
10,D,11
11,D,12
